# SQL Murder Mystery

## Can you find out whodunnit?

![A decorative illustration of a detective looking at an evidence board.](https://mystery.knightlab.com/174092-clue-illustration.png)

There's been a Murder in SQL City! The SQL Murder Mystery is designed to be both a self-directed lesson to learn SQL concepts and commands and a fun game for experienced SQL users to solve an intriguing crime.

## SQL sleuths start here

A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a **​murder​**that occurred sometime on ​**Jan.15, 2018​** and that it took place in ​**SQL City​**. Start by retrieving the corresponding crime scene report from the police department’s database.

### Exploring the Database Structure

Experienced SQL users can often use database queries to infer the structure of a database. But each database system has different ways of managing this information. The SQL Murder Mystery is built using SQLite. Use this SQL command to find the tables in the Murder Mystery database.

Run this query to find the names of the tables in this database.

SQLite-specific tip: the `sqlite_master` table is SQLite's catalog of tables and schemas, and other databases expose metadata differently.

This command is specific to SQLite. For other databases, you'll have to learn their specific syntax.


In [1]:
# Install required packages
%pip install jupysql sqlalchemy pandas --quiet

# Load SQL magic
%load_ext sql

# Connect to the database
%sql sqlite:///sql-murder-mystery.db
#%config SqlMagic.style = 'table'

Note: you may need to restart the kernel to use updated packages.


Connecting to 'sqlite:///sql-murder-mystery.db'

In [2]:
%%sql
SELECT name
FROM sqlite_master

Running query in 'sqlite:///sql-murder-mystery.db'

name
crime_scene_report
drivers_license
facebook_event_checkin
interview
get_fit_now_member
sqlite_autoindex_get_fit_now_member_1
get_fit_now_check_in
solution
check_solution
income



Besides knowing the table names, you need to know how each table is structured. The way this works is also dependent upon which database technology you use. Here's how you do it with SQLite.

Run this query to find the structure of the `crime_scene_report` table

Change the value of 'name' to see the structure of the other tables you learned about with the previous query.


In [3]:
%%sql
SELECT sql
FROM sqlite_master
where name = 'crime_scene_report'

Running query in 'sqlite:///sql-murder-mystery.db'

sql
"CREATE TABLE crime_scene_report ( date integer, type text, description text, city text )"


In [9]:
%%sql
SELECT *
FROM crime_scene_report
WHERE city = 'SQL City'
AND type =  'murder'
AND date = 20180115

Running query in 'sqlite:///sql-murder-mystery.db'

date,type,description,city
20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


In [13]:
%%sql
SELECT *
FROM person
WHERE address_street_name = 'Franklin Ave' OR address_street_name = "Northwestern Dr"
ORDER BY address_number

Running query in 'sqlite:///sql-murder-mystery.db'

id,name,license_id,address_number,address_street_name,ssn
93947,Omer Andreoni,605718,13,Franklin Ave,322594288
18651,Carleen Etoll,356746,22,Franklin Ave,193369255
33793,Amado Mattan,161915,99,Franklin Ave,125205748
16371,Annabel Miller,490173,103,Franklin Ave,318771143
12207,Wilmer Wolever,509484,139,Franklin Ave,636825374
61001,Laurine Bousman,197150,247,Franklin Ave,431360364
89906,Kinsey Erickson,510019,309,Northwestern Dr,635287661
15171,Weldon Penso,336999,311,Northwestern Dr,131379495
37616,Katelyn Schwerin,430143,332,Franklin Ave,392365033
95119,Hong Lisa,825828,375,Franklin Ave,113438176


In [17]:
%%sql 
SELECT *
FROM interview
WHERE person_id = 16371

Running query in 'sqlite:///sql-murder-mystery.db'

person_id,transcript
16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


In [18]:
%%sql 
SELECT *
FROM get_fit_now_check_in
WHERE check_in_date  = 20180109

Running query in 'sqlite:///sql-murder-mystery.db'

membership_id,check_in_date,check_in_time,check_out_time
X0643,20180109,957,1164
UK1F2,20180109,344,518
XTE42,20180109,486,1124
1AE2H,20180109,461,944
6LSTG,20180109,399,515
7MWHJ,20180109,273,885
GE5Q8,20180109,367,959
48Z7A,20180109,1600,1730
48Z55,20180109,1530,1700
90081,20180109,1600,1700



### The rest is up to you!

If you're really comfortable with SQL, you can probably get it from here. To help, here is the schema diagram:

![schema diagram](schema.png)

Use your knowledge of the database schema and SQL commands to find out who committed the murder.
### Check your solution

Did you find the killer? When you think you know the answer, submit your suspect using the following code and find out if you're right.


In [ ]:
%%sql
INSERT INTO solution VALUES (1, 'Insert the name of the person you found here');
SELECT value FROM solution;

Running query in 'sqlite:///sql-murder-mystery.db'

1 rows affected.

value
That's not the right person. Try again!
